In [107]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
from datetime import datetime, UTC
from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

current_dir = Path(os.getcwd()).resolve()
if current_dir.name == "LLMPolReasonEval": # uruchomione w Jupyter Lab
    project_root = current_dir
else:  # uruchomione w PyCharm
    project_root = current_dir.parents[2]
print(f"Project root: {project_root}")

sys.path.append(str(project_root / "src"))
from llm_pol_reason_eval.question_processing.dataset_manager import DatasetManager

if load_dotenv(os.path.join(project_root, '.env')):
    print (f"Loaded environment variables from {project_root / '.env'}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project root: C:\Users\piotr\PycharmProjects\LLMPolReasonEval
Loaded environment variables from C:\Users\piotr\PycharmProjects\LLMPolReasonEval\.env


In [108]:
hf_token = os.getenv("HF_TOKEN")
if not hf_token:
    raise ValueError("Brak tokena Hugging Face. Dodaj HF_TOKEN do pliku .env.")

In [109]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA not enabled")

NVIDIA GeForce GTX 1660 Ti


In [110]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Pamięć podręczna CUDA została wyczyszczona.")

Pamięć podręczna CUDA została wyczyszczona.


In [111]:
%%time
model_name = "speakleash/Bielik-1.5B-v3.0-Instruct"
print(f"Pobieranie modelu {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model załadowany na urządzenie: {device} z typem danych: {model.dtype}")


Pobieranie modelu speakleash/Bielik-1.5B-v3.0-Instruct...
Model załadowany na urządzenie: cuda z typem danych: torch.float16
CPU times: total: 8.69 s
Wall time: 3.08 s


In [112]:
%%time
prompt = "Jakie są główne cechy modelu Bielik 1.5B?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs["input_ids"], max_length=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Odpowiedź modelu:")
print(response)

Odpowiedź modelu:
Jakie są główne cechy modelu Bielik 1.5B?

Model Bielik 1.5B to zaawansowany model symulacyjny, który oferuje wiele funkcji i możliwości. Oto główne cechy tego modelu:

1. **Wysoka dokładność**: Model Bielik 1.5B jest zaprojektowany z myślą o wysokiej dokładności i realistycznym odwzorowaniu zachowań fizycznych.
2. **Różnorodność scenariuszy**: Model Bielik 
CPU times: total: 5.33 s
Wall time: 5.42 s


In [113]:
from os import getcwd
getcwd()

'C:\\Users\\piotr\\PycharmProjects\\LLMPolReasonEval'

In [114]:
json_file_path = project_root / "data" / "dataset" / "mvp_dataset_2025-06-08T20-42-43Z.json"
print(f"{json_file_path=}")
dataset_manager = DatasetManager()
dataset_manager.add_data_from_json_file(json_file_path)

json_file_path=WindowsPath('C:/Users/piotr/PycharmProjects/LLMPolReasonEval/data/dataset/mvp_dataset_2025-06-08T20-42-43Z.json')


In [115]:
len(dataset_manager.contexts)

31

In [116]:
len(dataset_manager.questions)

63

In [117]:
for question_key, question in dataset_manager.questions.items():
    print(f"{question_key=}")


question_key='EPOP-P1-100-2305_zad_1'
question_key='EPOP-P1-100-2305_zad_10'
question_key='EPOP-P1-100-2305_zad_11'
question_key='EPOP-P1-100-2305_zad_12'
question_key='EPOP-P1-100-2305_zad_13_temat_1'
question_key='EPOP-P1-100-2305_zad_13_temat_2'
question_key='EPOP-P1-100-2305_zad_13_temat_3'
question_key='EPOP-P1-100-2305_zad_2'
question_key='EPOP-P1-100-2305_zad_3'
question_key='EPOP-P1-100-2305_zad_4'
question_key='EPOP-P1-100-2305_zad_5'
question_key='EPOP-P1-100-2305_zad_6'
question_key='EPOP-P1-100-2305_zad_7'
question_key='EPOP-P1-100-2305_zad_8'
question_key='EPOP-P1-100-2305_zad_9_1'
question_key='EPOP-P1-100-2305_zad_9_2'
question_key='EPOP-P1-100-A-2405_zad_1'
question_key='EPOP-P1-100-A-2405_zad_10'
question_key='EPOP-P1-100-A-2405_zad_11'
question_key='EPOP-P1-100-A-2405_zad_12_temat_1'
question_key='EPOP-P1-100-A-2405_zad_12_temat_2'
question_key='EPOP-P1-100-A-2405_zad_12_temat_3'
question_key='EPOP-P1-100-A-2405_zad_2'
question_key='EPOP-P1-100-A-2405_zad_3_1'
questio

In [118]:
dataset_manager.questions["EPOP-P1-100-2305_zad_1"]["question_text"]

'Oceń prawdziwość podanych stwierdzeń odnoszących się do tekstu Józefa Tischnera. Zaznacz P, jeśli stwierdzenie jest prawdziwe, albo F - jeśli jest fałszywe.'

In [119]:
dataset_manager.get_stats()["question_category_by_type_stats"]

[(('matura_język_polski', 'open_text'), 38),
 (('matura_język_polski', 'closed_MTF'), 10),
 (('matura_język_polski', 'open_essay'), 4),
 (('matura_język_polski', 'closed_MCQ'), 3),
 (('matura_język_polski', 'open_summary'), 2),
 (('matura_język_polski', 'open_poetry_interpretation'), 2),
 (('matura_język_polski', 'closed_MRQ'), 2),
 (('matura_język_polski', 'open_synthesis'), 2)]

In [120]:
question_categories = set()
question_types = set()

for question_data in dataset_manager.questions.values():
    if "category" in question_data:
        question_categories.add(question_data["category"])
    if "question_type" in question_data:
        question_types.add(question_data["question_type"])

list_of_categories = list(question_categories)
list_of_question_types = list(question_types)

print(f"Dostępne kategorie pytań: {list_of_categories}")
print(f"Dostępne typy pytań: {list_of_question_types}")

Dostępne kategorie pytań: ['matura_język_polski']
Dostępne typy pytań: ['open_summary', 'closed_MRQ', 'open_essay', 'closed_MCQ', 'open_poetry_interpretation', 'closed_MTF', 'open_text', 'open_synthesis']


In [121]:
dataset_manager.get_question_category_list()

['matura_język_polski']

In [122]:
dataset_manager.get_question_type_list()

['closed_MCQ',
 'closed_MRQ',
 'closed_MTF',
 'open_essay',
 'open_poetry_interpretation',
 'open_summary',
 'open_synthesis',
 'open_text']

In [123]:
polski_open_text_questions = dataset_manager.get_questions_in_batches_as_jsonl_string(
    batch_size=10,
    query=lambda q: q.get("category") == 'matura_język_polski' and q.get("question_type") == 'open_text'
)

In [124]:
polski_open_text_questions

['{"questions": {"EPOP-P1-100-2305_zad_10": {"question_id": "EPOP-P1-100-2305_zad_10", "category": "matura_język_polski", "question_type": "open_text", "origin_source_id": "EPOP-P1-100-2305", "context_ids": ["EPOP-P1-100-2305_text_2"], "question_text": "Określ funkcję, jaką pełni w tekście Tomasza Kozłowskiego stosowanie czasowników w pierwszej osobie liczby mnogiej.", "generated_by": "Gemini Pro 2.5 Preview", "generation_date": "2025-06-05T01:14:19Z", "choices": [], "answer": {"correct_answer": null, "example_answers": ["skrócenie dystansu pomiędzy autorem a czytelnikiem ", "poświadczenie powszechności procesów/zjawisk, o których pisze autor ", "zbudowanie wspólnoty pomiędzy autorem a czytelnikiem ", "obiektywizacja wypowiedzi "], "statement_evaluations": [], "scoring_rules": {"scoring_summary": "1 pkt - poprawne określenie funkcji. 0 pkt - odpowiedź niepoprawna albo brak odpowiedzi. ", "evaluation_rules": [{"rule_id": "correct_function_identification", "description": "Ocena poprawnoś

In [125]:
import json

for i, batch_json in enumerate(polski_open_text_questions):
    batch_data = json.loads(batch_json)
    question_count = len(batch_data.get("questions", {}))

    context_count = len(batch_data.get("contexts", {}))

    byte_size = len(batch_json.encode('utf-8'))

    print(f"Batch {i+1}:")
    print(f"  - liczba pytań: {question_count}")
    print(f"  - liczba kontekstów: {context_count}")
    print(f"  - rozmiar: {byte_size} bajtów")

Batch 1:
  - liczba pytań: 10
  - liczba kontekstów: 4
  - rozmiar: 35429 bajtów
Batch 2:
  - liczba pytań: 10
  - liczba kontekstów: 7
  - rozmiar: 41001 bajtów
Batch 3:
  - liczba pytań: 10
  - liczba kontekstów: 10
  - rozmiar: 43052 bajtów
Batch 4:
  - liczba pytań: 8
  - liczba kontekstów: 8
  - rozmiar: 35889 bajtów


In [126]:
batch_data = json.loads(polski_open_text_questions[0])

In [127]:
first_key = list(batch_data["questions"].keys())[0]
first_question = batch_data["questions"].get(first_key, {})
question_text = first_question.get("question_text", "Brak tekstu pytania")
question_text

'Określ funkcję, jaką pełni w tekście Tomasza Kozłowskiego stosowanie czasowników w pierwszej osobie liczby mnogiej.'

In [128]:
question_context_ids = first_question.get("context_ids", [])
question_context_ids

['EPOP-P1-100-2305_text_2']

In [129]:
question_answer = first_question.get("answer", "Brak odpowiedzi")
question_answer

{'correct_answer': None,
 'example_answers': ['skrócenie dystansu pomiędzy autorem a czytelnikiem ',
  'poświadczenie powszechności procesów/zjawisk, o których pisze autor ',
  'zbudowanie wspólnoty pomiędzy autorem a czytelnikiem ',
  'obiektywizacja wypowiedzi '],
 'statement_evaluations': [],
 'scoring_rules': {'scoring_summary': '1 pkt - poprawne określenie funkcji. 0 pkt - odpowiedź niepoprawna albo brak odpowiedzi. ',
  'evaluation_rules': [{'rule_id': 'correct_function_identification',
    'description': 'Ocena poprawności określenia funkcji.',
    'element_max_points_contribution': None,
    'scoring_levels': [{'condition_or_level_description': 'Poprawne określenie funkcji.',
      'score_impact': 1},
     {'condition_or_level_description': 'Odpowiedź niepoprawna albo brak odpowiedzi.',
      'score_impact': 0}]}],
  'overall_guidance_for_llm': None},
 'max_points': 1,
 'external_context_required': None,
 'exam_requirements': [{'general_requirements': '1. Odbiór wypowiedzi i wy

In [130]:
for context_id in first_question.get("context_ids", []):
    context = batch_data["contexts"].get(context_id, {})
    context_text = context.get("context_content", "Brak tekstu kontekstu")
    print(f"Kontekst {context_id}: {context_text}")

Kontekst EPOP-P1-100-2305_text_2: Tomasz Kozłowski

Od Homo sapiens do Homo videns

Giovanni Sartori, włoski socjolog badający wpływ mediów na współczesne
społeczeństwa, ukuł i spopularyzował termin Homo videns, który moglibyśmy przetłumaczyć
jako „człowiek oglądający". Sens tych słów jest jednak znacznie szerszy, nie chodzi bowiem
tylko o samo stwierdzenie faktu, że oto dobrnęliśmy do takiego stadium ewolucji
biokulturowej, w którym coś lub kogoś nieustannie oglądamy. Jesteśmy raczej istotą nawykłą
do życia w rzeczywistości utkanej z obrazów i innych przekazów wizualnych. Obrazy lęgną
się w naszej pamięci i świadomości. Jesteśmy nauczeni myślenia obrazami i przetwarzania
ich. Są one podstawą naszych procesów psychicznych, a i procesy te dostosowały się
w pewnym sensie do otaczającej nas zewsząd obrazkowej rzeczywistości, wypierając
jednocześnie inne - być może bardziej tradycyjne - formy rozumowania. Przyznać trzeba, że Sartori w swoich twierdzeniach ma wiele racji - w istocie, myślen